# Projet Machine Learning for Time Series : ROCKET

## Etudiants : Lila Roig & Eva Robillard

Dans ce Notebook, nous étendons l'algorithme Rocket à la tache de prédiction. Rocket a initialement été créé et testé sur des taches de classification de séries temporelles univariées.

## Biblio

**Papiers**

- Rocket paper : https://arxiv.org/pdf/1910.13051

- Rocket official code : https://github.com/angus924/rocket

- Comment préparer les Séries temporelles : https://openreview.net/pdf?id=wEc1mgAjU-

- MiniRocket paper : https://arxiv.org/pdf/2012.08791

- MiniRocket official code : https://github.com/angus924/minirocket/tree/main

- MiniRocket tuto prise en main : https://github.com/timeseriesAI/tsai/blob/main/tutorial_nbs/10_Time_Series_Classification_and_Regression_with_MiniRocket.ipynb

- MultiRocket paper : https://arxiv.org/abs/2102.00457

---
**Classification Dataset**

**UCR Archive** (dataset sur lequel a été testé Rocket dans le papier original)

- UCR archive utilisée pour "entraîner" Rocket :  https://www.cs.ucr.edu/~eamonn/time_series_data_2018/ (répo officiel) et  http://www.timeseriesclassification.com/index.php (répo regroupant UCR acrhives et d'autres méthodes)

- UCR archive paper : https://arxiv.org/pdf/1810.07758

---
**Prediction Dataset**

**Monash Time Series Forecasting Archive** (dataset que nous allons utiliser pour tester Rocket sur la prédiction)

- Monash Dataset : https://forecastingdata.org/ (JE ME SUIS ARRETEE A BITCOIN INCLU)
- Monach paper dataset : https://openreview.net/pdf?id=wEc1mgAjU-
- Monach github : https://github.com/rakshitha123/TSForecasting?tab=readme-ov-file
- Monash Kaggle : https://www.kaggle.com/datasets/konradb/monash-time-series-forecasting-repository
- Monash Hugging Face : https://huggingface.co/datasets/Monash-University/monash_tsf
- Comment preprocesser les données : https://arxiv.org/pdf/1909.00590



**Attention !**

Dans le papier du Monash dataset, ils ne traitent pas dans leur test les datasets suivants pour des raisons de coût de calcul. Comme nous n'avons pas de machine très puissante non plus, nous ne traiterons pas non plus les datasets de l'archive Monash ci-dessous :
- **London smart meters**,
- **wind farms**,
- **solar power**,
- **wind power**
- **Kaggle web traffic daily**
- **solar 10 minutely**


## Mise en place

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
PATH = "/content/drive/My Drive/projet_time_series/"
import os
os.chdir(PATH)

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pickle
import math

from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

## Test de Rocket pour la regression


## Utilities

### Rocket algorithm


ROCKET attend en entrée plusieurs séries temporelles univariées sous la forme d'une matrice 2D. Il peut être étendu pour traiter plusieurs séries temporelles multivariées

> Texte ci-dessous issu de Rocket paper : https://arxiv.org/pdf/1910.13051

ROCKET transforms time series using convolutional kernels as found in typical convolutional neural networks. The parameters of these kernels are randomly selected as follows:

- **Length:** The length of the kernel is randomly selected from $\{7, 9, 11\}$ with equal probability, ensuring kernels are significantly shorter than input time series in most cases.
    
- **Weights:** The weights $w \in W$ are sampled from a normal distribution $w \sim \mathcal{N}(0, 1)$ and are mean-centered: $\omega = W - \bar{W}$. This ensures that most weights remain relatively small, though larger magnitudes are possible.
    
- **Bias:** The bias $b$ is sampled from a uniform distribution $b \sim \mathcal{U}(-1, 1)$. Only positive values are used in the feature maps. The bias effectively shifts the feature maps above or below zero, emphasizing different aspects of the input data.
    
- **Dilation:** Dilation is sampled exponentially as $d = 2^x$, where $x \sim \mathcal{U}(0, A)$ and $A = \log_2\left(\frac{l_{\text{input}} - 1}{l_{\text{kernel}} - 1}\right)$. This ensures the effective kernel length matches the input time series while allowing kernels to capture patterns at various scales.

- **Padding:** Padding is randomly applied with equal probability. If padding is used, zero-padding is added to the start and end of the series so that the kernel is centered at every point. Without padding, kernels focus on central patterns and may miss patterns at the edges.

**Stride** is always set to one. Nonlinearities such as **ReLU are not applied**, ensuring the resulting feature maps are agnostic to specific transformations. Input time series are assumed to be **normalized** with a mean of zero and a standard deviation of one.

These kernel parameters were determined to optimize classification accuracy on development datasets and generalize well to unseen data.


> Code ci-dessous issu de : https://github.com/angus924/rocket/blob/master/code/rocket_functions.py

In [ ]:
# ROCKET ALGORITHM
###############################
# Angus Dempster, Francois Petitjean, Geoff Webb
#
# @article{dempster_etal_2020,
#   author  = {Dempster, Angus and Petitjean, Fran\c{c}ois and Webb, Geoffrey I},
#   title   = {ROCKET: Exceptionally fast and accurate time classification using random convolutional kernels},
#   year    = {2020},
#   journal = {Data Mining and Knowledge Discovery},
#   doi     = {https://doi.org/10.1007/s10618-020-00701-z}
# }
#
# https://arxiv.org/abs/1910.13051 (preprint)

import numpy as np
from numba import njit, prange

@njit("Tuple((float64[:],int32[:],float64[:],int32[:],int32[:]))(int64,int64)")
def generate_kernels(input_length, num_kernels):
    """
    Generates random convolutional kernels as described in the ROCKET paper

    Parameters:
    - input_length: Length of the input time series
    - num_kernels: Number of random kernels to generate

    Returns:
    - weights: Flattened array of kernel weights
    - lengths: Array of kernel lengths (randomly chosen from {7, 9, 11})
    - biases: Array of biases (random values in [-1, 1])
    - dilations: Array of dilation values (sampled on an exponential scale)
    - paddings: Array of padding values (randomly applied)
    """

    # Candidate kernel lengths (ROCKET uses lengths {7, 9, 11})
    candidate_lengths = np.array((7, 9, 11), dtype = np.int32)
    lengths = np.random.choice(candidate_lengths, num_kernels)

    # Initialize arrays for kernel parameters
    weights = np.zeros(lengths.sum(), dtype = np.float64)  # Kernel weights
    biases = np.zeros(num_kernels, dtype = np.float64)     # Kernel biases
    dilations = np.zeros(num_kernels, dtype = np.int32)    # Kernel dilations
    paddings = np.zeros(num_kernels, dtype = np.int32)     # Kernel paddings

    a1 = 0  # Pointer for the start of each kernel's weights

    for i in range(num_kernels):

        _length = lengths[i]  # Length of the current kernel

        # Generate weights from a normal distribution (mean 0, std 1)
        _weights = np.random.normal(0, 1, _length)

        b1 = a1 + _length
        weights[a1:b1] = _weights - _weights.mean()  # Center the weights (mean = 0)

        # Generate bias from a uniform distribution in [-1, 1]
        biases[i] = np.random.uniform(-1, 1)

        # Compute dilation on an exponential scale
        dilation = 2 ** np.random.uniform(0, np.log2((input_length - 1) / (_length - 1)))
        dilation = np.int32(dilation)
        dilations[i] = dilation

        # Randomly apply padding (0 or centered padding)
        padding = ((_length - 1) * dilation) // 2 if np.random.randint(2) == 1 else 0
        paddings[i] = padding

        a1 = b1  # Move pointer to the next kernel's weights

    return weights, lengths, biases, dilations, paddings

@njit(fastmath = True)
def apply_kernel(X, weights, length, bias, dilation, padding):
    """
    Applies a single convolutional kernel to a time series

    In the code, X represents the input time series, but it is treated as a
    2D matrix, which means it can include univariate or multivariate time series,
    depending on its structure.

    Parameters:
    - X: The input time series of size
    - weights: Weights of the kernel
    - length: Length of the kernel
    - bias: Bias of the kernel
    - dilation: Dilation factor
    - padding: Padding size

    Returns:
    - ppv: Proportion of positive values in the feature map
    - max: Maximum value in the feature map
    """

    input_length = len(X)

    # Compute the output length of the feature map
    output_length = (input_length + (2 * padding)) - ((length - 1) * dilation)

    _ppv = 0  # Proportion of positive values
    _max = np.NINF  # Maximum value in the feature map (initialized to negative infinity)

    # Define the range for convolution
    end = (input_length + padding) - ((length - 1) * dilation)

    for i in range(-padding, end):  # Iterate over the time series with padding

        _sum = bias  # Initialize convolution result with the bias

        index = i

        for j in range(length):  # Apply the kernel

            if index > -1 and index < input_length:  # Check if the index is within bounds
                _sum = _sum + weights[j] * X[index]  # Convolve with weights

            index = index + dilation  # Apply dilation

        if _sum > _max:  # Update maximum value
            _max = _sum

        if _sum > 0:  # Count positive values for ppv
            _ppv += 1

    # Return the proportion of positive values and the maximum
    return _ppv / output_length, _max


@njit("float64[:,:](float64[:,:],Tuple((float64[::1],int32[:],float64[:],int32[:],int32[:])))", parallel = True, fastmath = True)
def apply_kernels(X, kernels):
    """
    Applies all generated kernels to a batch of time series

    In the code, X represents the input time series, but it is treated as a
    2D matrix, which means it can include univariate or multivariate time series,
    depending on its structure.

    Parameters:
    - X: A batch of input time series (2D array)
    - kernels: Tuple containing kernel parameters

    Returns:
    - _X: A transformed feature matrix (2 features per kernel: ppv and max)
   """

    weights, lengths, biases, dilations, paddings = kernels

    num_examples, _ = X.shape
    num_kernels = len(lengths)

    # Initialize the output feature matrix
    _X = np.zeros((num_examples, num_kernels * 2), dtype = np.float64)  # Two features per kernel: ppv and max

    for i in prange(num_examples):  # Parallelize over examples

        a1 = 0  # Pointer for weights
        a2 = 0  # Pointer for features

        for j in range(num_kernels):

            b1 = a1 + lengths[j]
            b2 = a2 + 2

            # Apply a single kernel and store the resulting features
            _X[i, a2:b2] = \
            apply_kernel(X[i], weights[a1:b1], lengths[j], biases[j], dilations[j], paddings[j])

            a1 = b1  # Move pointer to the next kernel's weights
            a2 = b2  # Move pointer to the next feature set

    return _X



**Forme de X :**

$X$ est une matrice 2D où :

- Les lignes correspondent aux exemples ou au nombre de séries temporelles dans le dataset (num_examples). Chaque exemple correspond à une série temporelle différente.

- Les colonnes correspondent aux points temporels (input_length)

**ST univariés :** $X$ doit avoir la forme :

`X.shape = (num_examples, input_length)`

**ST multivariées :** $X$ doit avoir la forme :

`X.shape = (num_examples, num_variables, input_length)`

> Actuellement Rocket ne supporte pas les Séries Temporelles Multivariées, mais il est possible de l'adapter pour ce cas.

---

**Comment adapter ROCKET au multivarié**

Il est possible d'adapter l'algorithme ROCKET pour les séries temporelles multivariées. Pour ce faire, nous pouvons utiliser la méthode décrite dans le papier MiniROCKET (https://arxiv.org/pdf/2012.08791) qui est une amélioration de ROCKET. Cette méthode consiste à appliquer les mêmes noyaux de convolution indépendamment à chaque variable d'une série multivariée.

Cette approche permet d'extraire les caractéristiques de chaque variable individuellement, sans essayer d'exploiter directement les corrélations entre les variables.
Cette indépendance facilite l'application sur des Séries Temporelles Multivariées, en particulier lorsque les variables sont hétérogènes (par exemple, des capteurs avec des unités ou des dynamiques différentes). \

Nous avons donc implémenté cette approche avec la fonction `apply_kernels_multivariate`.

Cependant, cette approche entraîne une augmentation linéaire du nombre de features calculées par ROCKET :

`Taille totale des features  = num_kernels * 2 * num_variables`

Et ceci devient ingérable en mémoire pour de grands datasets.

Comme cette approche multivariée de ROCKET applique les mêmes noyaux de convolution de façon indépendante sur chaque variable, une approche équivalente à laquelle nous avons pensé est d'appliquer ROCKET de manière totalement indépendante sur chaque variable univariée de la série temporelle multivariée. Cela revient exactement à ce que fait ROCKET multivarié, mais avec des avantages en termes de mémoire et de modularité.

Pour ce faire, nous suivrons les étapes suivantes:

Pour les séries temporelles **univariées**
- Générer des noyaux : Les noyaux sont générés pour chaque série univariée (avec generate_kernels).
- Appliquer ROCKET univarié pour chaque série univariée. Notons que les noyaux appliqués pour chaque série univariée sont différents d'une série à l'autre.

Pour les séries temporelles **multivariées**
- Générer une fois les noyaux : Les noyaux sont générés une seule fois (avec generate_kernels).
- Appliquer ROCKET univarié pour chaque variable : on applique les mêmes noyaux précedemment générés indépendamment à chaque variable de la série temporelle multivariée.
- Concaténer les résultats : Les features générées pour chaque variable sont concaténées pour obtenir un tableau final.


Enfin, notons que l'approche qui consiste à appliquer les mêmes noyaux indépendamment sur chaque variable d'une série temporelle multivariée ne permet pas de capturer les dépendances entre les variables. Pour palier ce problème, l'algorithme **MultiROCKET** a été développé (https://arxiv.org/abs/2102.00457). Nous ne sommes pas sûres d'avoir le temps de tester MultiRocket et si nous avons le temps, nous pourrions églement avoir des problèmes de d'explosion mémoire pour de grosses séries temporelles multivariées.


### Chargement archive Monash

In [ ]:
# Chargement des datasets de l'archive Monash
# Fonction issue de https://github.com/rakshitha123/TSForecasting?tab=readme-ov-file
######################################
from datetime import datetime
from distutils.util import strtobool
import pandas as pd

# Converts the contents in a .tsf file into a dataframe and returns it along with other meta-data of the dataset: frequency, horizon, whether the dataset contains missing values and whether the series have equal lengths
#
# Parameters
# full_file_path_and_name - complete .tsf file path
# replace_missing_vals_with - a term to indicate the missing values in series in the returning dataframe
# value_column_name - Any name that is preferred to have as the name of the column containing series values in the returning dataframe
def convert_tsf_to_dataframe(
    full_file_path_and_name,
    replace_missing_vals_with="NaN",
    value_column_name="series_value",
):
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, "r", encoding="cp1252") as file:
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"):  # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if (
                                len(line_content) != 3
                            ):  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                            if (
                                len(line_content) != 2
                            ):  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                            if line.startswith("@frequency"):
                                frequency = line_content[1]
                            elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                            elif line.startswith("@missing"):
                                contain_missing_values = bool(
                                    strtobool(line_content[1])
                                )
                            elif line.startswith("@equallength"):
                                contain_equal_length = bool(strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception(
                                "Missing attribute section. Attribute section must come before data."
                            )

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception(
                            "Missing attribute section. Attribute section must come before data."
                        )
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if len(series) == 0:
                            raise Exception(
                                "A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series. Missing values should be indicated with ? symbol"
                            )

                        numeric_series = []

                        for val in series:
                            if val == "?":
                                numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if numeric_series.count(replace_missing_vals_with) == len(
                            numeric_series
                        ):
                            raise Exception(
                                "All series values are missing. A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series."
                            )

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(
                                    full_info[i], "%Y-%m-%d %H-%M-%S"
                                )
                            else:
                                raise Exception(
                                    "Invalid attribute type."
                                )  # Currently, the code supports only numeric, string and date types. Extend this as required.

                            if att_val is None:
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        loaded_data = pd.DataFrame(all_data)

        return (
            loaded_data,
            frequency,
            forecast_horizon,
            contain_missing_values,
            contain_equal_length,
        )

# Example of usage
# loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe("TSForecasting/tsf_data/sample.tsf")

# print(loaded_data)
# print(frequency)
# print(forecast_horizon)
# print(contain_missing_values)
# print(contain_equal_length)

**Précisions sur l'archive Monash**

Dans l'archive Monash, les séries temporelles sont sous la forme suivante :

**Séries Univariées**
- Chaque fichier représente **plusieurs** séries tremporelles univariées.
  - Chaque ligne représente une série temporelle univariée
  - Les colonnes représentent un pas de temps.

**Séries Multivariées**
- Chaque fichier représente **une** série temporelle multivariée.
  - Lignes $T$ : Chaque ligne correspond à un pas de temps unique.
  - Colonnes $n$ : Chaque colonne correspond à une variable.

### Déterminer le Forecast Horizon

**Dans le papier du Monash dataset :**

Biblio pour cette fontion :
-  papier Monsah : https://openreview.net/pdf?id=wEc1mgAjU-

**Extrait du papier Monash (https://openreview.net/pdf?id=wEc1mgAjU-)** :

"Forecast horizons are chosen for each dataset to evaluate the model performance. For all competition datasets, we use the forecast horizons originally employed in the competitions. For the remaining datasets, 12 months ahead forecasts are obtained for monthly datasets, 8 weeks ahead forecasts are obtained for weekly datasets, except the solar weekly dataset, and 30 days ahead forecasts are obtained for daily datasets. For the solar weekly dataset, we use a horizon of 5 as the series in this dataset are relatively short compared with other weekly datasets. For half-hourly, hourly and other high-frequency datasets, we set the forecasting horizon to one week, e.g., 168 is used as the horizon for hourly datasets."


In [ ]:
def determine_forecast_horizon(frequency, dataset_name, forecast_horizon):
    """
    Détermine l'horizon temporel (forecast horizon) selon l'heuristique décrite pour les datasets du Monash Dataset.

    From the Monash Time Series Forecasting Archive Paper (https://openreview.net/pdf?id=wEc1mgAjU-):
    >>> Forecast horizons are chosen for each dataset to evaluate the model performance. For all competition datasets,
    we use the forecast horizons originally employed in the competitions. For the remaining datasets, 12 months ahead
    forecasts are obtained for monthly datasets, 8 weeks ahead forecasts are obtained for weekly datasets,
    except the solar weekly dataset, and 30 days ahead forecasts are obtained for daily datasets. For the solar
     weekly dataset, we use a horizon of 5 as the series in this dataset are relatively short compared with other
     weekly datasets. For half-hourly, hourly and other high-frequency datasets, we set the forecasting horizon
     to one week, e.g., 168 is used as the horizon for hourly datasets. <<<

    Parameters:
        frequency (str): Fréquence de la série temporelle (e.g., 'monthly', 'weekly', 'daily', 'hourly').
        dataset_name (str): Nom du dataset, utilisé pour des cas spécifiques (e.g., 'solar weekly').
        forecast_horizon (int): Horizon temporel fourni directement. Si None, l'horizon est déterminé automatiquement.

    Returns:
        int: L'horizon temporel pour le dataset donné.
    """
    # Si l'horizon est déjà fourni, on le retourne directement
    if forecast_horizon is not None:
        return forecast_horizon

    # Heuristiques selon la fréquence
    frequency_to_horizon = {
        'yearly' : 1,        # 1 an
        'quaterly' : 4,      # 4 trimestres (1 an)
        'monthly': 12,       # 12 mois (1 an)
        'weekly': 8,         # 8 semaines
        'daily': 30,         # 30 jours
        'hourly': 168,       # 1 semaine = 168 heures
        'half_hourly': 336,  # 1 semaine = 336 demi-heures
        '10-minutely': 1008, # 1 semaine = 1008 intervalles de 10 minutes
    }

    # Exception pour le dataset "solar weekly"
    if "solar_weekly" in dataset_name.lower():
        return 5

    # Vérifier si la fréquence est dans les règles
    if frequency in frequency_to_horizon:
        return frequency_to_horizon[frequency]

    # Si la fréquence n'est pas reconnue
    raise ValueError(f"Fréquence inconnue ou non supportée : {frequency}")

### Déterminer les lags (= taille fenêtre glissante)

**Dans le papier du Monash dataset** :

Biblio pour cette fontion :
-  https://openreview.net/pdf?id=wEc1mgAjU-
-  https://openreview.net/pdf?id=wEc1mgAjU-

Les valeurs retardées (ou "lags") correspondent aux observations passées utilisées par le modèle pour faire des prédictions futures.

Donc :  lag = k = taille de la fenêtre glissante

**Extrait du papier Monash (https://openreview.net/pdf?id=wEc1mgAjU-)**:

The number of lagged values used in the global models are determined based on a heuristic suggested in prior work [59]. Generally, the number of lagged values is chosen as the seasonality multiplied with 1.25. If the datasets contain short series and it is impossible to use the above defined number of lags, for example in the Dominick and solar weekly datasets, then the number of lagged values is chosen as the forecast horizon multiplied with 1.25, assuming that the horizon is not arbitrarily chosen but based on certain characteristics of the time series structure. When defining the number of lagged values for multi-seasonal datasets, we consider the corresponding weekly seasonality value, e.g., 168 for hourly datasets. If it is impossible to use the number of lagged values obtained with the weekly seasonality due to high memory and computational requirements, for example with the traffic hourly and electricity hourly datasets, then we use the corresponding daily seasonality value to define the number of lags, e.g., 24 for hourly datasets. In particular, due to high memory and computational requirements, the number of lagged values is chosen as 50 for the solar 10 minutely dataset which is less than the above mentioned heuristics based on seasonality and forecasting horizon suggest.


**Extrait du papier : https://arxiv.org/pdf/1909.00590** :

Putting these ideas together, we select the input window size m with two options. One is to make the input window size slightly bigger than the output window size (m = 1.25 ∗ output window size). The other is to make the input window size slightly bigger than the seasonality period (m = 1.25 ∗ seasonality period). For instance if the time series has weekly seasonality (seasonality period = 7) with expected forecasting horizon 56, with the first option we set the input window size to be 70 (1.25 ∗ 56), whereas with the second option it is set to be 9 (1.25 ∗ 7). The constant 1.25 is selected purely as a heuristic. The idea is to ascertain that every recurrent cell instance at each time step gets to see at least its last periodic cycle so that it can model any remaining stochastic seasonality. In case, the total length of the time series is too short, m is selected to be either of the two feasible, or some other possible value if none of them work. For instance, the forecasting horizon 6 subgroup of the CIF 2016 dataset comprises of such short series and m is selected to be 7 (slightly bigger than the output window size) even though the seasonality period is equal to 12.


In [ ]:
def determine_lag(frequency, forecast_horizon, dataset_name, series_length, multiply_by=1):
    """
    Détermine le nombre de lags selon l'heuristique décrite dans le papier du Monash Dataset.
    Si le lag calculé est plus petit que l'horizon de prévision, il est ajusté à forecast_horizon.
    Les lags correspondent aux observations passées utilisées par le modèle pour faire des prédictions futures.
    Si on a une série temporelle Yt​, les lagged values sont des décalages temporels de cette série :
    Y_{t−1}, Y_{t−2},...,Y_{t−p}​, où p est le nombre de lagged values.

    La taille de la fenêtre glissante peut être fixée égale au nombre de lagged values :
    Window Size = Number of Lagged Values
    Cela garantit que chaque fenêtre contiendra suffisamment d'informations pour que
    le modèle capture les motifs temporels pertinents.

    From the Monash Time Series Forecasting Archive Paper (https://openreview.net/pdf?id=wEc1mgAjU-):
    >>> The number of lagged values used in the global models are determined based on a heuristic suggested in
    prior work [59]. Generally, the number of lagged values is chosen as the seasonality multiplied with 1.25.
    If the datasets contain short series and it is impossible to use the above defined number of lags, for
     example in the Dominick and solar weekly datasets, then the number of lagged values is chosen as the
     forecast horizon multiplied with 1.25, assuming that the horizon is not arbitrarily chosen but based on
     certain characteristics of the time series structure. When defining the number of lagged values for
     multi-seasonal datasets, we consider the corresponding weekly seasonality value, e.g., 168 for hourly
     datasets. If it is impossible to use the number of lagged values obtained with the weekly seasonality
     due to high memory and computational requirements, for example with the traffic hourly and electricity
     hourly datasets, then we use the corresponding daily seasonality value to define the number of lags,
    e.g., 24 for hourly datasets. In particular, due to high memory and computational requirements, the
    number of lagged values is chosen as 50 for the solar 10 minutely dataset which is less than the above
    mentioned heuristics based on seasonality and forecasting horizon suggest. <<<

    Parameters:
        frequency (str): Fréquence de la série temporelle (e.g., 'monthly', 'weekly', 'daily', 'hourly').
        forecast_horizon (int): Horizon de prévision.
        dataset_name (str, optional): Nom du dataset, utilisé pour des cas spécifiques (e.g., 'solar weekly').
        series_length (int, optional): Longueur totale de la série. Nécessaire pour vérifier les contraintes.
        multiply_by (int, optional): Facteur de multiplication du lag.

    Returns:
        lag (int): Le lag pour le dataset donné.
        seasonality (int) : saisonnalité du dataset donné.
    """
    # Saisonnalités définies par fréquence
    frequency_to_seasonality = {
        'yearly': 1,
        'quarterly': 4,
        'monthly': 12,
        'weekly': 7,
        'daily': 7,
        'hourly': 168,
        'half_hourly': 336,
        '10-minutely': 1008,
    }

    # Exception pour des cas spécifiques
    if "solar_weekly" in dataset_name.lower():
        return math.ceil(5 * 1.25)  # Spécifique à ce dataset

    # Obtenir la saisonnalité
    seasonality = frequency_to_seasonality.get(frequency, None)
    if seasonality is None:
        raise ValueError(f"Fréquence inconnue ou non supportée : {frequency}")

    # Calculer les deux options pour le lag
    option1 = math.ceil(forecast_horizon * 1.25)
    option2 = math.ceil(seasonality * 1.25)

    # Choisir la meilleure option selon la longueur de la série
    if series_length is not None and series_length < option2:
        lag = option1
    else:
        lag = option2

    # Contraintes mémoire pour certains datasets spécifiques
    if "traffic hourly" in dataset_name.lower() or "electricity hourly" in dataset_name.lower():
        lag = min(lag, math.ceil(24 * 1.25))
    if "10-minutely" in dataset_name.lower():
        lag = 50  # Fixe pour ce dataset

    # Ajuster pour garantir m >= forecast_horizon
    lag = max(lag, forecast_horizon)

    return lag, seasonality

### Mise en forme du jeu de données

**Mettre en forme le jeu de données**

Si data est univarié alors data["series_value"] est de la forme : (num_examples, time_length)
c'est à dire composé de plusieurs ST univariées. Chaque ligne correspond à une ST univariée
et chaque colonne a un pas de temps.

Si data est multivarié alors data["series_value"] est de la forme : (time_length, num_variables)
c'est à dire composé UNE seule ST multivariée. Chaque ligne correspond à un pas de temps et
et chaque colonne a une variable. Afin d'appliquer les mêmes noyaux de ROCKET indépendament
sur chaque variable, on transpose data["series_value"] pour qu'il soit de la forme
(num_variables, time_length). Ainsi, une ST multivariée est vue comme plusieurs ST univariées.
On ne prends pas en compte ici la dépendace entre les variables.

In [ ]:
def preprocess_series(data, dataset_name):
    """
    Crée le jeu de données contenant les séries tremporelles preprocessed.
    Préprocesser les séries temporelles selon qu'elles soient univariées ou multivariées
    avec normalisation des données et gestion des longueurs variable.
    Chaque ST univariée est normalisée indépendamment.
    De même, chaque variable de la ST multivariée est normalisée indépendamment.

    Parameters:
        data (pandas.DataFrame): DataFrame contenant une colonne "series_value" qui stocke les séries temporelles.
            - Si univarié : "series_value" est de la forme (num_examples, time_length) (longueurs variables possibles).
            - Si multivarié : "series_value" est de la forme (time_length, num_variables) (longueurs variables possibles).
        dataset_name (str): Nom du dataset, utilisé pour déterminer s'il est univarié ou multivarié.

    Returns:
        list of np.ndarray ou np.ndarray:
            - Si univarié : Liste de tableaux NumPy normalisés pour chaque série (indépendamment).
              Chaque tableau peut avoir une longueur différente.
              Chaque élément de la liste contient une ST univariée.
            - Si multivarié : Tableau NumPy normalisé (num_variables, time_length),
              chaque variable étant normalisée indépendamment.
              Chaque élément de la liste contient une variable de la ST multivariée.
    """
    series_list = data["series_value"].to_list()

    if 'univariate' in dataset_name:
        # Données univariées : Normalisation série par série
        processed_series = [ # ST univariée : (num_examples, time_length)
            StandardScaler().fit_transform(np.array(serie).reshape(-1, 1)).flatten()
            for serie in series_list
        ]
        return np.array(processed_series, dtype=object)  # Retourne une liste de séries normalisées (longueurs variables possibles)

    elif 'multivariate' in dataset_name:
        # Données multivariées
        mts = np.array(series_list, dtype=object)  # Tableau des séries multivariées

        # Vérifier si les longueurs des séries sont homogènes
        if all(len(serie) == len(mts[0]) for serie in mts):
            # Transposer directement le tableau entier si toutes les longueurs sont homogènes
            mts = np.array(series_list).T  # Transpose de (time_length, num_variables) à (num_variables, time_length)
            scaler = StandardScaler()
            processed_series = np.array([scaler.fit_transform(var.reshape(-1, 1)).flatten() for var in mts])
            return processed_series
        else:
            # Si les longueurs sont inhomogènes, traiter chaque série séparément
            processed_series = []
            for serie in mts:
                transposed = np.array(serie).T  # Transposer la série multivariée
                processed_series.append(
                    [StandardScaler().fit_transform(var.reshape(-1, 1)).flatten() for var in transposed]
                )
            return processed_series  # Retourne une liste de séries normalisées

    else:
        raise ValueError("Le type de dataset doit être spécifié dans le nom : 'univariate' ou 'multivariate'.")


### Vérifier la taille de la série et adapter la taille de fenêtre en fonction

In [ ]:
def check_and_adjust_series(series, window_size, forecast_horizon, stride, min_window_size, min_stride=1):
    """
    Combine la vérification de la taille de l'ensemble d'entraînement avec l'ajustement dynamique des paramètres de fenêtres glissantes.

    Parameters:
        series (list or np.array): La série temporelle à analyser.
        window_size (int): Taille initiale de la fenêtre glissante.
        forecast_horizon (int): Horizon de prédiction.
        stride (int): Pas entre les fenêtres glissantes.
        min_window_size (int): Taille minimale de la fenêtre glissante.
        min_stride (int): Stride minimal autorisé.

    Returns:
        dict: Contient les résultats :
            - 'is_valid' (bool): True si les paramètres sont valides.
            - 'adjusted_window_size' (int): Taille de fenêtre ajustée.
            - 'adjusted_stride' (int): Stride ajusté.
            - 'n_windows' (int): Nombre de fenêtres glissantes possibles.
            - 'min_required_length' (int): Longueur minimale requise pour l'ensemble d'entraînement.
            - 'reason' (str): Raison si la série est invalide.
    """
    series_length = len(series)

    # Ajuster la taille de la fenêtre et le stride
    adjusted_stride = max(min_stride, stride)
    adjusted_window_size = max(min_window_size, min(window_size, series_length // 2))

    # Taille minimale requise pour générer des fenêtres valides
    min_required_length = adjusted_window_size + forecast_horizon + adjusted_stride - 1

    # Taille de l'ensemble d'entraînement
    T_train = series_length - forecast_horizon

    # Vérification si l'ensemble d'entraînement est suffisamment long
    if T_train < min_required_length:
        return {
            "is_valid": False,
            "adjusted_window_size": adjusted_window_size,
            "adjusted_stride": adjusted_stride,
            "n_windows": 0,
            "min_required_length": min_required_length,
            "reason": f"Training set too short: T_train={T_train} < min_required_length={min_required_length}."
        }

    # Calculer le nombre de fenêtres glissantes possibles
    n_windows = max(0, (series_length - adjusted_window_size - forecast_horizon) // adjusted_stride + 1)

    # Vérification de validité finale
    if n_windows < 2:
        return {
            "is_valid": False,
            "adjusted_window_size": adjusted_window_size,
            "adjusted_stride": adjusted_stride,
            "n_windows": n_windows,
            "min_required_length": min_required_length,
            "reason": f"Not enough windows (n_windows={n_windows} < 2)."
        }

    # Si tout est valide, retourner les paramètres ajustés
    return {
        "is_valid": True,
        "adjusted_window_size": adjusted_window_size,
        "adjusted_stride": adjusted_stride,
        "n_windows": n_windows,
        "min_required_length": min_required_length,
        "reason": None
    }

### Division en train et test


Biblio pour cette fontion :
-  https://openreview.net/pdf?id=wEc1mgAjU-

In [ ]:
def split_time_series(series, forecast_horizon):
    """
    Divise une série temporelle en ensembles d'entraînement et de test.

    Parameters:
        series (list or np.ndarray): Série temporelle à diviser.
        forecast_horizon (int): Taille de la partie forecast (H).

    Returns:
        train (np.ndarray): Partie entraînement (l - H).
        test (np.ndarray): Partie validation (H).
    """
    series = np.asarray(series)
    train = series[:-forecast_horizon]
    test = series[-forecast_horizon:]
    return train, test

### Application des fenêtres glissantes

**Mise en forme des données pour le forecasting avec des fenêtres glissantes et un stride.**

Biblio:
- https://github.com/timeseriesAI/tsai/blob/main/tsai/data/preparation.py
- https://towardsdatascience.com/fast-and-robust-sliding-window-vectorization-with-numpy-3ad950ed62f5
-  https://openreview.net/pdf?id=wEc1mgAjU-



In [ ]:
def sliding_window(series, window_size, forecast_horizon, stride=1, fill_incomplete=True):
    """
    Crée un jeu de données pour le forecasting avec des fenêtres glissantes et un stride.

    Parameters:
        series (np.ndarray or list): Série temporelle univariée de forme (T,).
        window_size (int): Taille de la fenêtre d'entrée (W).
        forecast_horizon (int): Horizon de prévision (H).
        stride (int): Pas de décalage entre les fenêtres (par défaut : 1).
        fill_incomplete (bool): Si True, remplit les fenêtres de y incomplètes avec la dernière valeur disponible.

    Returns:
        X (np.ndarray): Fenêtres d'entrée de forme (N, W).
        y (np.ndarray): Fenêtres de sortie de forme (N, H).
    """
    # Convertir la série en tableau NumPy si ce n'est pas déjà fait
    series = np.asarray(series)
    T = len(series)  # Nombre de pas de temps (T)

    # Vérifier que la fenêtre glissante est au moins aussi grande que l'horizon de prévision
    if window_size < forecast_horizon:
        raise ValueError("La taille de la fenêtre (window_size) ne peut pas être plus petite que l'horizon temporel (forecast_horizon).")

    # Calculer le nombre total de fenêtres
    n_windows = (T - window_size - forecast_horizon) // stride + 1
    if n_windows <= 0:
        raise ValueError("Taille de la série trop courte pour les paramètres donnés.")

    # Générer les indices pour les fenêtres d'entrée (X)
    input_indices = np.arange(window_size) + np.arange(0, n_windows * stride, stride)[:, None]
    input_indices = input_indices.astype(int)

    X = series[input_indices]  # Fenêtres d'entrée : (N, W)

    # Générer les indices pour les fenêtres de sortie (y)
    output_indices = np.arange(forecast_horizon) + np.arange(0, n_windows * stride, stride)[:, None] + window_size

    # Traiter les indices de sortie incomplets si fill_incomplete est activé
    y = []
    for indices in output_indices:
        if indices[-1] < T:  # Cas où y est complet
            y.append(series[indices])
        else:  # Cas où y est incomplet
            incomplete_y = series[indices[indices < T]].tolist()
            last_value = incomplete_y[-1] if incomplete_y else series[-1]
            filled_y = incomplete_y + [last_value] * (forecast_horizon - len(incomplete_y))
            y.append(filled_y)

    y = np.array(y)  # Fenêtres de sortie : (N, H)

    return X, y

### Reconstruire la série à partir des fenêtres glissante

In [ ]:
def inverse_sliding_window(X, y, window_size, forecast_horizon, stride=1):
    """
    Reconstruit une série temporelle à partir des fenêtres glissantes.

    Parameters:
        X (np.ndarray): Fenêtres d'entrée de forme (N, W).
        y (np.ndarray): Fenêtres de sortie de forme (N, H).
        window_size (int): Taille de la fenêtre d'entrée (W).
        forecast_horizon (int): Horizon de prévision (H).
        stride (int): Pas de décalage entre les fenêtres (par défaut : 1).

    Returns:
        series_reconstructed (np.ndarray): Série temporelle reconstruite.
    """
    # Dimensions des fenêtres
    n_windows, W = X.shape
    H = y.shape[1]

    # Taille estimée de la série reconstruite
    total_length = (n_windows - 1) * stride + window_size + forecast_horizon
    series_reconstructed = np.zeros(total_length)
    weights = np.zeros(total_length)  # Poids pour moyenne des recouvrements

    for i in range(n_windows):
        start_x = i * stride
        end_x = start_x + window_size
        series_reconstructed[start_x:end_x] += X[i]
        weights[start_x:end_x] += 1

        start_y = end_x
        end_y = start_y + forecast_horizon
        series_reconstructed[start_y:end_y] += y[i]
        weights[start_y:end_y] += 1

    # Normaliser par les poids pour moyenne dans les zones de recouvrement
    weights[weights == 0] = 1  # Éviter la division par zéro
    series_reconstructed /= weights

    return series_reconstructed

### Entraîner un régresseur après ROCKET


In [ ]:
def train_and_predict_model(X_train_features, y_train, X_test_features, model_name='ridge', **kwargs):
    """
    Entraîne un modèle de régression spécifié et effectue une prédiction.

    Parameters:
        X_train_features (np.ndarray): Caractéristiques d'entraînement (features) issues de ROCKET.
        y_train (np.ndarray): Labels ou cibles d'entraînement.
        X_test_features (np.ndarray): Caractéristiques de test (features) issues de ROCKET.
        model_name (str): Le nom du modèle à utiliser. Options disponibles :
                          - 'ridge' pour RidgeCV (régression Ridge).
                          - 'lasso' pour LassoCV (régression Lasso).
                          - 'elasticnet' pour ElasticNetCV (régression ElasticNet).
                          - 'svr' pour SVR (Support Vector Regression).
                          - 'random_forest' pour RandomForestRegressor (régression Random Forest).
                          - 'xgboost' pour XGBRegressor (régression XGBoost).
        **kwargs: Paramètres supplémentaires spécifiques au modèle choisi.

    Returns:
        tuple:
            - y_pred (np.ndarray): Prédictions effectuées par le modèle.
            - regressor (object): Instance du modèle entraîné.

    Avantages et inconvénients des modèles :
    - RidgeCV: L2 regularization. Convient aux datasets avec de nombreuses features corrélées.
                Pas de sparsité dans les coefficients.
    - LassoCV: L1 regularization. Encourage la sparsité et sélectionne les features importantes.
                Peut ignorer certaines features pertinentes si elles sont corrélées.
    - ElasticNetCV: Combine L1 et L2. Bon compromis entre sélection de features et généralisation.
                     Convient si beaucoup de features sont corrélées.
    - SVR: Bonne performance avec des noyaux non linéaires comme RBF. Nécessite une mise à l'échelle des données.
           Peut être lent pour de grands datasets.
    - RandomForestRegressor: Manipule bien de nombreuses features. Robuste au surapprentissage.
                             Moins interprétable et plus coûteux en mémoire.
    - XGBRegressor: Performant même avec des déséquilibres ou de nombreuses features inutiles.
                    Nécessite un ajustement précis des hyperparamètres pour éviter le surapprentissage.
    """
    if model_name == 'ridge':
        # Ridge Regression (L2 Regularization): Gère les features corrélées sans sparsité
        regressor = RidgeCV(alphas=np.logspace(-3, 3, 7), **kwargs)
        regressor.fit(X_train_features, y_train)

    elif model_name == 'lasso':
        # Lasso Regression (L1 Regularization): Encourage la sparsité
        regressor = LassoCV(alphas=np.logspace(-3, 3, 7), cv=5, **kwargs)
        regressor.fit(X_train_features, y_train)

    elif model_name == 'elasticnet':
        # ElasticNet (L1 + L2 Regularization): Combine sparsité et robustesse
        regressor = ElasticNetCV(l1_ratio=0.5, alphas=np.logspace(-3, 3, 7), cv=5, **kwargs)
        regressor.fit(X_train_features, y_train)

    elif model_name == 'svr':
        # Support Vector Regression (SVR): Bon pour des relations complexes, mais lent sur de grands datasets
        regressor = SVR(kernel='linear', C=1.0, **kwargs)
        regressor.fit(X_train_features, y_train)

    elif model_name == 'random_forest':
        # Random Forest: Gère de nombreuses features, robuste, mais moins interprétable
        regressor = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, **kwargs)
        regressor.fit(X_train_features, y_train)

    elif model_name == 'xgboost':
        # XGBoost: Puissant et performant, mais nécessite un ajustement d'hyperparamètres
        regressor = XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.1, **kwargs)
        regressor.fit(X_train_features, y_train)

    else:
        raise ValueError(f"Modèle inconnu : {model_name}. Options disponibles : 'ridge', 'lasso', 'elasticnet', 'svr', 'random_forest', 'xgboost'.")

    # Effectuer les prédictions sur les données de test
    y_pred = regressor.predict(X_test_features)

    return y_pred, regressor


### Calcul du MASE

Le **modèle naïf** est une approche de prévision très simple, souvent utilisée comme référence ou benchmark minimal pour évaluer les performances d'autres modèles.

*Principe* : La prochaine valeur de la série temporelle est égale à la dernière valeur observée.
On suppose que la série ne présente pas de tendance ni de saisonnalité significative.
Formule : $\hat{y}_{t+h} = y_t$

Le **modèle snaïve** (saisonnier naïf) est une extension du modèle naïf classique qui exploite la saisonnalité présente dans une série temporelle. Plutôt que de prédire uniquement en fonction de la dernière observation, il répète les valeurs des dernières périodes de saisonnalité.

*Principe* : La série temporelle est saisonnière, avec une période connue ($s$). Les valeurs futures suivent le même motif que les valeurs des cycles précédents.
Formule : $\hat{y}_{t+h} = y_{t+h-s}$

Le **Mean Absolute Scaled Error (MASE)** est défini comme suit :

$$
\text{MASE} = \frac{\text{MAE du modèle}}{\text{MAE du modèle naïf}}
$$

- **MAE du modèle**: Moyenne des erreurs absolues entre les prédictions $\hat{y}$ et les valeurs réelles $y$.
- **MAE du modèle naïf** : Moyenne des erreurs absolues pour le modèle naïf sur la série d'entraînement. Le modèle naïf utilise la dernière observation pour prédire la suivante :
$$
\text{MAE naïf} = \frac{1}{T-1} \sum_{t=2}^{T} |y_t - y_{t-1}|
$$

Interprétation :
- $MASE < 0.5 $ : Le modèle est significativement meilleur que le modèle naïf.
- $0.5 < MASE < 1$ : Le modèle est légèrement meilleur que le modèle naîf, mais l'amélioration est modérée.  
- $MASE = 1$ : Le modèle né apporte aucune amélioration par rapport au modèle naïf.
- $MASE > 1$ : Le modèle est moins performant que le modèle naïf.

In [ ]:
def compute_mase(actual_values, predicted_values, training_series, seasonality, epsilon=1e-8):
    """
    Calcul du Mean Absolute Scaled Error (MASE) avec gestion des séries courtes.

    Parameters:
        actual_values (np.ndarray): Valeurs réelles (test) à prédire.
        predicted_values (np.ndarray): Valeurs prédites par le modèle.
        training_series (np.ndarray): Série temporelle d'entraînement utilisée pour calculer l'erreur naïve.
        seasonality (int): Saison (par exemple, 12 pour des données mensuelles avec une saison annuelle).
        epsilon (float): Petite valeur pour éviter la division par zéro.

    Returns:
        float: Valeur du MASE.
    """
    # Étape 1 : Calcul des prévisions naïves
    if len(training_series) < seasonality:
        # Si la série d'entraînement est plus courte que la saisonnalité, utiliser un modèle naïf basé sur un décalage minimal
        naive_forecasts = training_series[:-1]  # Décalage de 1 point
        naive_actuals = training_series[1:]    # Décalage de 1 point
    else:
        # Prévisions basées sur la saisonnalité
        naive_forecasts = training_series[:-seasonality]
        naive_actuals = training_series[seasonality:]

    # Étape 2 : Calcul des erreurs naïves
    naive_errors = np.abs(naive_actuals - naive_forecasts)
    if len(naive_errors) == 0:
        raise ValueError("Les erreurs naïves sont vides. Vérifiez `training_series` et `seasonality`.")

    # Étape 3 : Calcul de l'erreur absolue moyenne (MAE) du modèle naïf
    mae_naive = np.mean(naive_errors)
    if mae_naive < epsilon:
        print("La MAE du modèle naïf est trop faible, ce qui peut entraîner une division par zéro.")

    # Étape 4 : Calcul du MASE
    mase = np.mean(np.abs(actual_values - predicted_values)) / mae_naive

    return mase

### Calcul des autres métriques

In [ ]:
def calculate_metrics(y_true, y_pred, epsilon=1e-8):
    """
    Calcule les métriques : sMAPE, MSE, RMSE, msMAPE.

    Parameters:
        y_true (np.ndarray): Valeurs réelles.
        y_pred (np.ndarray): Valeurs prédites.
        epsilon (float): Petite valeur pour éviter la division par zéro.

    Returns:
        dict: Dictionnaire contenant les métriques calculées.
    """
    # Assurer que y_true et y_pred sont des tableaux numpy
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    # Calcul du sMAPE
    numerator = np.abs(y_true - y_pred)
    denominator = np.maximum(np.abs(y_true) + np.abs(y_pred), epsilon)
    smape = 100 * np.mean(numerator / (denominator / 2))

    # Calcul du MSE
    mse = np.mean((y_true - y_pred) ** 2)

    # Calcul du RMSE
    rmse = np.sqrt(mse)

    # Calcul du msMAPE
    msmapen = np.maximum(np.maximum(np.abs(y_true), np.abs(y_pred)), epsilon)
    msmape = np.mean(np.abs(y_true - y_pred) / msmapen)  # Moyenne des erreurs normalisées

    # Retourner les résultats sous forme de dictionnaire
    return {
        "sMAPE": smape,
        "MSE": mse,
        "RMSE": rmse,
        "msMAPE": msmape,
    }

### Sauver les résultats dans un excel

In [ ]:
def save_results_to_excel(file_path, data_dict):
    """
    Enregistre les résultats dans un fichier Excel, en ajoutant les nouvelles lignes au fichier existant.

    Parameters:
        file_path (str): Chemin vers le fichier Excel.
        data_dict (dict): Dictionnaire contenant les colonnes et leurs valeurs à enregistrer (clé = nom de colonne, valeur = liste).
    """
    # Convertir le dictionnaire en DataFrame
    df_to_save = pd.DataFrame(data_dict)

    # Si le fichier existe déjà, lire son contenu et ajouter les nouvelles lignes
    if os.path.exists(file_path):
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, df_to_save], ignore_index=True)
    else:
        # Si le fichier n'existe pas, créer un nouveau DataFrame
        updated_df = df_to_save

    # Sauvegarder dans le fichier Excel
    updated_df.to_excel(file_path, index=False)

## Appliquer ROCKET pour la prédiction

Dans la section ci-dessous, nous allons utiliser toutes les fonctions définies plus haut pour appliquer l'algorithme ROCKET pour la prédiction.

In [ ]:
import time
import sys

def apply_rocket_forecasting(path, ts_nature, num_kernels=10000,
                             verbose_all=False, verbose_special=False,
                             regressor_model='ridge', results_excel_path="results.xlsx"):
    """
    Applique ROCKET pour la tâche de forecasting sur plusieurs datasets univariés ou multivariés,
    calcule les métriques d'évaluation (MASE, MSE) et sauvegarde les résultats dans un fichier Excel.

    Parameters:
        path (str): Chemin de base contenant les datasets.
        ts_nature (str): Type de série temporelle. Peut être 'univariate' ou 'multivariate'.
        num_kernels (int, optional): Nombre de noyaux aléatoires à générer pour ROCKET. Par défaut, 10000.
        verbose_all (bool, optional): Indique si tous les messages d'information sont activés. Par défaut, False.
        verbose_special (bool, optional): Indique si les messages spéciaux sont activés
            (pour les séries temporelles ayant des problèmes particuliers). Par défaut, False.
        regressor_model (str, optional): Modèle de régression à utiliser ('ridge', 'xgboost', ou 'randomforest'). Par défaut, 'ridge'.
        results_excel_path (str, optional): Chemin du fichier Excel pour enregistrer les résultats. Par défaut, "results.xlsx".

    Returns:
        None. Les résultats sont sauvegardés dans un fichier Excel.
    """

    # Spécifier le chemin du dossier
    folder = 'tsf_data/' + ts_nature + '/'
    folder_path = path + folder
    if verbose_all: print(f"Folder path: {folder_path}")

    # Liste tous les fichiers et sous-dossiers dans le dossier
    items = os.listdir(folder_path)
    dataset_names = [f"{folder}{item}" for item in items]
    if verbose_all: print(f"Datasets found: {dataset_names}")

    # Initialiser un dictionnaire pour stocker les résultats au format requis pour l'enregistrement
    columns = ["is_multivariate", "dataset_name", "forecast_horizon", "frequency", "regressor",
               "mean_mase", "median_mase", "mean_mse", "median_mse", "mean_rmse", "median_rmse",
               "mean_smape", "median_smape", "mean_msmape", "median_msmape",
               "equal_length", "processing_time"]
    results_for_excel = {col: [] for col in columns}

    print("\nStart forecasting...")
    # DEBUT BOUCLE FOR 1 sur les datasets
    for j in range(len(dataset_names)):
        dataset_name = dataset_names[j]
        print("\nAnalyzing dataset", dataset_name)
        start_time = time.time()

        (data,
        frequency,
        forecast_horizon,
        contain_missing_values,
        contain_equal_length) = convert_tsf_to_dataframe(dataset_name,
                                                        replace_missing_vals_with="NaN",
                                                        value_column_name="series_value")
        if verbose_all: print(f"Dataset loaded. Frequency: {frequency}, Forecast Horizon: {forecast_horizon}")

        # Vérifier que le dataset ne contient pas de valeurs manquantes
        if contain_missing_values:
            raise ValueError(f"Dataset {dataset_name} contains missing values")

        # Déterminer l'horizon temporel pour ce dataset
        forecast_horizon = determine_forecast_horizon(frequency, dataset_name, forecast_horizon)
        if verbose_all: print(f"Determined forecast horizon: {forecast_horizon}")

        # Mise en forme du dataset et préprocessing
        series = preprocess_series(data, dataset_name)
        #   si data univarié : series = liste de taille (num_examples) où chaque élément
        #   est une ST univariée de taille (time_length)
        #   si data multivarié : series = liste de taille (num_variables) où chaque
        #   élément est une ST univariée de taille (time_length)

        # Initialiser des listes pour stocker les métriques pour ce dataset
        mase_list = []
        mse_list  = []
        rmse_list = []
        smape_list = []
        msmapen_list = []

        # Initialiser pour le calcul du temps restant
        series_start_time = time.time()

        # DEBUT BOUCLE FOR 2 sur les séries
        for i in range(len(series)):

            if verbose_all: print("")
            # ----- Affichage progression -----
            current_time = time.time()
            # Temps écoulé pour les séries déjà traitées
            elapsed_time = current_time - series_start_time
            # Temps moyen par série
            avg_time_per_series = elapsed_time / (i + 1)
            # Temps restant estimé
            remaining_time = avg_time_per_series * (len(series) - (i + 1))
            remaining_minutes, remaining_seconds = divmod(remaining_time, 60)
            # Affichage du compteur avec temps restant
            sys.stdout.write(
            f"\rProcessing series {i + 1}/{len(series)}... Estimated time left: {int(remaining_minutes)}m {int(remaining_seconds)}s"
            )
            sys.stdout.flush()
            # ---------------------------------
            # On itère sur les séries temporelles
            serie = series[i]

            # Diviser en train et en test
            train, test = split_time_series(serie, forecast_horizon)
            # Taille de `train`: (len(serie) - forecast_horizon,)
            # Taille de `test`: (forecast_horizon,)
            if verbose_all: print(f"Data split. Train size: {len(train)}, Test size: {len(test)}")

            # Suivant le répertoire dans lequel est classé la série (long ou short) appliquer un traitement différent
            # On applique un + grand stride et taille de fenêtre pour les séries longues car sinon le
            # code est trop long à tourner et peut faire exploser la RAM.
            if 'long' in ts_nature:
                multiply_by = 2
                stride = 2
            else:
                multiply_by = 1
                stride = 1

            # Calculer la taille des fenêtres glissantes (lag)
            min_window_size, seasonality = determine_lag(frequency, forecast_horizon, dataset_name=dataset_name, series_length=len(serie))
            window_size = min_window_size * multiply_by # si multiply_by = 1, min_window_size = window_size
            if verbose_all: print(f"\n\nmin_window_size : {forecast_horizon}, window_size : {window_size}, stride : {stride}, seasonality : {seasonality}")

            # Vérifier la taille des fenêtres glissantes et adapter si besoin
            check = check_and_adjust_series(train, window_size, forecast_horizon, stride, min_window_size, min_stride=1)

            # si la série est trop courte même après ajustement.
            if check['is_valid'] == False :
                if verbose_all or verbose_special : print(f"WARNING for series {i} : {check['reason']}. We pass this series.")
                continue  # Ignorer cette série

            # sinon ajuster les paramètres de la série :
            window_size = check['adjusted_window_size'] # taille de fenêtre ajustée.
            stride = check['adjusted_stride'] #stride ajusté
            if verbose_all: print(f"adjusted_window_size: {window_size}, adjusted_stride : {stride}")

            # Appliquer la fenêtre glissante
            X_train, y_train = sliding_window(train, window_size, forecast_horizon, stride, fill_incomplete=True)
            X_train = X_train.astype(np.float64)
            # Taille de `X_train`: (N, window_size) où N dépend de `train`, `window_size`, et `stride`
            # Taille de `y_train`: (N, forecast_horizon)
            if verbose_all: print(f"Sliding window generated. X shape: {X_train.shape}, y shape: {y_train.shape}")

            input_length = X_train.shape[1]  # Longueur de la série temporelle

            # Appliquer ROCKET
            if 'multivariate' in ts_nature and i == 0:
                kernels = generate_kernels(input_length, num_kernels)

            if 'univariate' in ts_nature:
                kernels = generate_kernels(input_length, num_kernels)

            # Extraire les caractéristiques
            X_train_features = apply_kernels(X_train, kernels)
            # Taille de `X_train_features`: (N, n_features) où `n_features` dépend du nombre de noyaux ROCKET
            if verbose_all: print(f"Features extracted. Train features shape: {X_train_features.shape}")

            if verbose_all:
                # Reconstruire la série à partir des fenêtres d'entraînement (facultatif, pour validation)
                train_reconstructed = inverse_sliding_window(X_train, y_train, window_size, forecast_horizon, stride)
                # Taille de `train_reconstructed`: dépend de `X_train` et de `y_train`, approximativement égale à `train`
                # Comparer la reconstruction avec le train original
                reconstruction_valid = np.allclose(train[:len(train_reconstructed)], train_reconstructed, atol=1e-6)
                if reconstruction_valid:
                    print("Reconstruction 'inverse sliding windows' is correct.")
                else:
                    print("Reconstruction 'inverse sliding windows' is incorrect. Verify sliding windows or parameters.")

            # Préparer les données de test
            X_test = train[-window_size:].reshape(1, -1)  # Dernières valeurs du train comme entrée
            X_test = X_test.astype(np.float64) # Taille de `X_test`: (1, window_size)
            if verbose_all: print(f"Test data prepared. X shape: {X_test.shape}")

            X_test_features = apply_kernels(X_test, kernels) # Taille de `X_test_features`: (1, n_features)
            if verbose_all: print(f"Features extracted. Test features shape: {X_test_features.shape}")

            # Appliquer un régresseur
            if np.var(y_train) < 1e-8:  # Vérifie si y_train est constant
                # Méthode de prédiction triviale si y_train constant
                constant_prediction = np.mean(y_train)  # Moyenne de y_train
                y_test_pred = np.full((1, forecast_horizon), constant_prediction)  # Remplit le forecast_horizon avec la constante
                if verbose_all or verbose_special:
                    print(f"y_train variance too low. Using constant prediction. Constant prediction made: {constant_prediction}")
            else:
                # Entraînement du modèle si y_train n'est pas constant
                y_test_pred, regressor = train_and_predict_model(X_train_features, y_train, X_test_features, model_name=regressor_model)
                y_test_pred = y_test_pred.flatten()
                if verbose_all : print(f"Model trained. Predictions shape: {y_test_pred.shape}")

            if verbose_all: print(f"actual values : {test}\npredicted_values : {y_test_pred}")

            # Calculer les métriques d'évaluation
            mase = compute_mase(test, y_test_pred, train, seasonality)
            metrics = calculate_metrics(test, y_test_pred, epsilon=1e-8)
            if verbose_all : print(f"MASE: {mase}, MSE: {metrics['MSE']}, RMSE : {metrics['RMSE']}, sMAPE : {metrics['sMAPE']} , msMAPE : {metrics['msMAPE']}" )

            # ajouter ces métriques à la liste
            mase_list.append(mase)
            mse_list.append(metrics['MSE'])
            rmse_list.append(metrics['RMSE'])
            smape_list.append(metrics['sMAPE'])
            msmapen_list.append(metrics['msMAPE'])

        # FIN BOUCLE FOR 2

        # Calculer les moyennes des métriques pour ce dataset
        mean_mase = np.mean(mase_list)
        mean_mse = np.mean(mse_list)
        mean_rmse = np.mean(rmse_list)
        mean_smape = np.mean(smape_list)
        mean_msmape = np.mean(msmapen_list)
        median_mase = np.median(mase_list)
        median_mse = np.median(mse_list)
        median_rmse = np.median(rmse_list)
        median_smape = np.median(smape_list)
        median_msmape = np.median(msmapen_list)

        # Calcul du temps total nécessaire pour traiter une série temporelle
        processing_time = time.time() - start_time
        processing_minutes = int(processing_time // 60)  # Minutes entières
        processing_seconds = int(processing_time % 60)  # Secondes restantes
        formatted_processing_time = f"{processing_minutes} min {processing_seconds} sec"

        # Ajouter les résultats pour ce dataset au dictionnaire
        results_for_excel["is_multivariate"].append(ts_nature == "multivariate")
        results_for_excel["dataset_name"].append(dataset_name)
        results_for_excel["forecast_horizon"].append(forecast_horizon)
        results_for_excel["frequency"].append(frequency)
        results_for_excel["regressor"].append(regressor_model)

        results_for_excel["mean_mase"].append(mean_mase)
        results_for_excel["mean_mse"].append(mean_mse)
        results_for_excel["mean_rmse"].append(mean_rmse)
        results_for_excel["mean_smape"].append(mean_smape)
        results_for_excel["mean_msmape"].append(mean_msmape)

        results_for_excel["median_mase"].append(median_mase)
        results_for_excel["median_mse"].append(median_mse)
        results_for_excel["median_rmse"].append(median_rmse)
        results_for_excel["median_smape"].append(median_smape)
        results_for_excel["median_msmape"].append(median_msmape)

        results_for_excel["equal_length"].append(contain_equal_length)
        results_for_excel["processing_time"].append(formatted_processing_time)

        # Sauvegarder les résultats dans l'Excel à chaque dataset
        save_results_to_excel(results_excel_path, results_for_excel)

    # FIN BOUCLE FOR 1
    print("\n...done!")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
#warnings.resetwarnings()

In [ ]:
apply_rocket_forecasting(path=PATH,
                         ts_nature='univariate/dataset_short/finish_test',
                         num_kernels=1000, # 10000 est le chiffre recommandé dans le Papier Rocket
                         verbose_all=False,
                         verbose_special=False,
                         regressor_model='ridge',
                         results_excel_path="results.xlsx")


Start forecasting...

Analyzing dataset tsf_data/univariate/dataset_short/finish_test/vehicle_trips_dataset_without_missing_values.tsf
Processing series 100/329... Estimated time left: 0m 17sLa MAE du modèle naïf est trop faible, ce qui peut entraîner une division par zéro.
Processing series 267/329... Estimated time left: 0m 3sLa MAE du modèle naïf est trop faible, ce qui peut entraîner une division par zéro.
Processing series 284/329... Estimated time left: 0m 2sLa MAE du modèle naïf est trop faible, ce qui peut entraîner une division par zéro.
Processing series 329/329... Estimated time left: 0m 0s
...done!


In [ ]:
apply_rocket_forecasting(path=PATH,
                         ts_nature='univariate/dataset_long',
                         num_kernels=1000, # 10000 est le chiffre recommandé dans le Papier Rocket
                         verbose_all=False,
                         verbose_special=False,
                         regressor_model='ridge',
                         results_excel_path="results.xlsx")


Start forecasting...

Analyzing dataset tsf_data/univariate/dataset_long/m4_yearly_dataset.tsf
Processing series 23000/23000... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_long/m4_quarterly_dataset.tsf
Processing series 24000/24000... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_long/m4_monthly_dataset.tsf
Processing series 48000/48000... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_long/m4_daily_dataset.tsf
Processing series 4227/4227... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_long/m4_hourly_dataset.tsf
Processing series 414/414... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_long/kdd_cup_2018_dataset_without_missing_values.tsf
Processing series 270/270... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_long/weather_dataset.tsf
Processing series 409/3010... Estimated time left: 1266m 50s

KeyboardInterrupt: 

In [ ]:
apply_rocket_forecasting(path=PATH,
                         ts_nature='multivariate/test',
                         num_kernels=1000, # 10000 est le chiffre recommandé dans le Papier Rocket
                         verbose_all=False,
                         verbose_special=False,
                         regressor_model='ridge',
                         results_excel_path="results.xlsx")


Start forecasting...

Analyzing dataset tsf_data/multivariate/test/solar_weekly_dataset.tsf
Processing series 1/52... Estimated time left: 0m 0s

TypeError: cannot unpack non-iterable int object

In [ ]:
apply_rocket_forecasting(path=PATH,
                         ts_nature='univariate/dataset_short',
                         num_kernels=1000, # 10000 est le chiffre recommandé dans le Papier Rocket
                         verbose_all=False,
                         verbose_special=False,
                         regressor_model='xgboost',
                         results_excel_path="results.xlsx")


Start forecasting...

Analyzing dataset tsf_data/univariate/dataset_short/m3_quarterly_dataset.tsf
Processing series 756/756... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_short/tourism_yearly_dataset.tsf
Processing series 518/518... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_short/tourism_quarterly_dataset.tsf
Processing series 427/427... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_short/tourism_monthly_dataset.tsf
Processing series 118/366... Estimated time left: 555m 12s

KeyboardInterrupt: 

In [ ]:
apply_rocket_forecasting(path=PATH,
                         ts_nature='univariate/dataset_short',
                         num_kernels=1000, # 10000 est le chiffre recommandé dans le Papier Rocket
                         verbose_all=False,
                         verbose_special=False,
                         regressor_model='random_forest',
                         results_excel_path="results.xlsx")


Start forecasting...

Analyzing dataset tsf_data/univariate/dataset_short/m3_quarterly_dataset.tsf
Processing series 756/756... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_short/tourism_yearly_dataset.tsf
Processing series 518/518... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_short/tourism_quarterly_dataset.tsf
Processing series 427/427... Estimated time left: 0m 0s
Analyzing dataset tsf_data/univariate/dataset_short/tourism_monthly_dataset.tsf
Processing series 37/366... Estimated time left: 103m 45s

KeyboardInterrupt: 